# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseProject.db')
df = pd.read_sql_table('DisasterResponseProject', engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Define url pattern
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(x).lower().strip() for x in tokens]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in STOPWORDS]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)
values = []
for i in range(len(y_test.columns)):
      
        values.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
        precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
        recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')])
        
values = pd.DataFrame(values, columns=['f1_score', 'precision', 'recall'], index = y_test.columns)
        
print(values)

                        f1_score  precision    recall
related                 0.795545   0.795545  0.795545
request                 0.887855   0.887855  0.887855
offer                   0.996033   0.996033  0.996033
aid_related             0.745346   0.745346  0.745346
medical_help            0.925694   0.925694  0.925694
medical_products        0.950259   0.950259  0.950259
search_and_rescue       0.973604   0.973604  0.973604
security                0.981691   0.981691  0.981691
military                0.967653   0.967653  0.967653
child_alone             1.000000   1.000000  1.000000
water                   0.947055   0.947055  0.947055
food                    0.930272   0.930272  0.930272
shelter                 0.935459   0.935459  0.935459
clothing                0.983522   0.983522  0.983522
money                   0.974977   0.974977  0.974977
missing_people          0.987489   0.987489  0.987489
refugees                0.965517   0.965517  0.965517
death                   0.96

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    "clf__estimator__max_depth": [1, 3],
    "clf__estimator__n_estimators": [50, 100]
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1, cv=5, verbose=2)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total=  24.9s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   40.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total=  24.6s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total=  25.0s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total=  24.8s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total=  24.9s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=100 ...
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=100, total=  31.0s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=100 ...
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=100, total=  31.2s
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=100 ...
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=100, total=  30.

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 15.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__max_depth': [1, 3], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred_grid = cv.predict(X_test)
values = []
for i in range(len(y_test.columns)):

        values.append([f1_score(y_test.iloc[:, i].values, y_pred_grid[:, i], average='micro'),
        precision_score(y_test.iloc[:, i].values, y_pred_grid[:, i], average='micro'),
        recall_score(y_test.iloc[:, i].values, y_pred_grid[:, i], average='micro')])
        
values = pd.DataFrame(values, columns=['f1_score', 'precision', 'recall'], index = y_test.columns)
        
print(values)

                        f1_score  precision    recall
related                 0.756790   0.756790  0.756790
request                 0.831096   0.831096  0.831096
offer                   0.996033   0.996033  0.996033
aid_related             0.587122   0.587122  0.587122
medical_help            0.924779   0.924779  0.924779
medical_products        0.948734   0.948734  0.948734
search_and_rescue       0.973146   0.973146  0.973146
security                0.981843   0.981843  0.981843
military                0.966128   0.966128  0.966128
child_alone             1.000000   1.000000  1.000000
water                   0.935459   0.935459  0.935459
food                    0.887092   0.887092  0.887092
shelter                 0.913030   0.913030  0.913030
clothing                0.982301   0.982301  0.982301
money                   0.974214   0.974214  0.974214
missing_people          0.987336   0.987336  0.987336
refugees                0.964602   0.964602  0.964602
death                   0.95

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
# AdaBoost Classifier:
pipeline_ada = Pipeline(steps=[
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(AdaBoostClassifier()))
])


# Training pipeline:
pipeline_ada.fit(X_train, y_train)

# Metrics
y_pred_ada = pipeline_ada.predict(X_test)
values = []
for i in range(len(y_test.columns)):

        values.append([f1_score(y_test.iloc[:, i].values, y_pred_ada[:, i], average='micro'),
        precision_score(y_test.iloc[:, i].values, y_pred_ada[:, i], average='micro'),
        recall_score(y_test.iloc[:, i].values, y_pred_ada[:, i], average='micro')])
        
values = pd.DataFrame(values, columns=['f1_score', 'precision', 'recall'], index = y_test.columns)
        
print(values)

                        f1_score  precision    recall
related                 0.771285   0.771285  0.771285
request                 0.896094   0.896094  0.896094
offer                   0.995117   0.995117  0.995117
aid_related             0.757858   0.757858  0.757858
medical_help            0.929051   0.929051  0.929051
medical_products        0.956973   0.956973  0.956973
search_and_rescue       0.973604   0.973604  0.973604
security                0.980012   0.980012  0.980012
military                0.972383   0.972383  0.972383
child_alone             1.000000   1.000000  1.000000
water                   0.961703   0.961703  0.961703
food                    0.945682   0.945682  0.945682
shelter                 0.942783   0.942783  0.942783
clothing                0.987336   0.987336  0.987336
money                   0.975435   0.975435  0.975435
missing_people          0.988404   0.988404  0.988404
refugees                0.966585   0.966585  0.966585
death                   0.96

### 9. Export your model as a pickle file

In [12]:
# save the model to disk
pickle.dump(pipeline_ada, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.